## Métadonnées

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

# --- 1. Chemin du fichier ---
nc_file = r"C:\_GLOBAL\R_D\hackathon_mf\data\tasmaxAdjust_FR-Metro_CNRM-ESM2-1_ssp585_r1i1p1f2_CNRM-MF_CNRM-ALADIN64E1_v1-r1_MF-CDFt-ANASTASIA-SAFRAN-1985-2014_day_20150101-21001231.nc"

# --- 2. Ouvrir le fichier ---
ds = xr.open_dataset(nc_file)
# --- 3. Afficher les informations générales ---
print("\n=== Informations du dataset ===")
print(ds)

# --- 4. Lister les variables ---
print("\n=== Variables disponibles ===")
print(list(ds.variables))

# --- 5. Afficher les dimensions ---
print("\n=== Dimensions ===")
print(ds.dims)

# --- 6. Attributs globaux ---
print("\n=== Attributs globaux ===")
print(ds.attrs)

# --- 7. Exemple : explorer la variable 'huss' s’il existe ---
if "huss" in ds.variables:
    var = ds["huss"]
    print("\n=== Informations sur la variable 'huss' ===")
    print(var)

    # Exemple de statistiques simples
    print("\nValeur min :", float(var.min()))
    print("Valeur max :", float(var.max()))
    print("Moyenne :", float(var.mean()))

    # --- 8. Visualisation d'une carte pour la première date ---
    var.isel(time=0).plot()
    plt.title("Humidité spécifique (huss) – première date")
    plt.show()

else:
    print("La variable 'huss' n'existe pas dans ce fichier.")


## Traitements !

#### Traitement des données de température afin d'avoir un nombre de jours à > 35°C par an par période TRACC

In [ ]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
from rasterio.features import rasterize
from shapely.geometry import mapping


# ===========================================================
# 0) LISTE DES TRAITEMENTS À EFFECTUER
# ===========================================================
jobs = [
    {
        "nc_file": r"C:\_GLOBAL\R_D\hackathon_mf\data\models\tasmaxAdjust_FR-Metro_CNRM-ESM2-1_historical_r14i1p1f2_CNRM-MF_CNRM-ALADIN64E1_v1-r1_MF-CDFt-ANASTASIA-SAFRAN-1985-2014_day_19500101-20141231.nc",
        "var_name": "tasmaxAdjust",
        "threshold": 308.15,  # 35°C en Kelvin
        "mod_name": "mean_hot_days_per_year_1950_2014",
        "periods": {
            "1985-2014": ("1985-01-01", "2014-12-31")
        }
    },
        {
        "nc_file": r"C:\_GLOBAL\R_D\hackathon_mf\data\models\tasmaxAdjust_FR-Metro_CNRM-ESM2-1_ssp370_r14i1p1f2_CNRM-MF_CNRM-ALADIN64E1_v1-r1_MF-CDFt-ANASTASIA-SAFRAN-1985-2014_day_20150101-21001231.nc",
        "var_name": "tasmaxAdjust",
        "threshold": 308.15,  # 35°C en Kelvin
        "mod_name": "mean_hot_days_per_year_ssp370_2015_2100",
        "periods": {
            "2037-2056": ("2037-01-01", "2056-12-31"),
            "2063-2082": ("2063-01-01", "2082-12-31"),
        }
    },
{
        "nc_file": r"C:\_GLOBAL\R_D\hackathon_mf\data\models\tasmaxAdjust_FR-Metro_CNRM-ESM2-1_ssp585_r1i1p1f2_CNRM-MF_CNRM-ALADIN64E1_v1-r1_MF-CDFt-ANASTASIA-SAFRAN-1985-2014_day_20150101-21001231.nc",
        "var_name": "tasmaxAdjust",
        "threshold": 308.15,  # 35°C en Kelvin
        "mod_name": "mean_hot_days_per_ssp585_year_2015_2100",
        "periods": {
            "2032-2051": ("2032-01-01", "2051-12-31"),
            "2052-2061": ("2052-01-01", "2061-12-31"),
            "2072-2091": ("2072-01-01", "2091-12-31"),
        }
    },
]

# couches géographiques
gpkg_file = r"C:\_GLOBAL\R_D\hackathon_mf\data\supra_admin_20251202_093135.gpkg"
import fiona

layers = fiona.listlayers(gpkg_file)

# ===========================================================
# 1) BOUCLE GENERALE SUR LES FICHIERS
# ===========================================================
all_results = []

for job in jobs:
    print("\n===================================================")
    print(f"📌 Traitement fichier : {job['nc_file']}")
    print("===================================================")

    nc_file   = job["nc_file"]
    var_name  = job["var_name"]
    threshold = job["threshold"]
    periods   = job["periods"]
    mod_name  = job["mod_name"]

    # -----------------------------
    # CHARGEMENT NETCDF
    # -----------------------------
    ds = xr.open_dataset(nc_file)
    da = ds[var_name]

    transform = ds.rio.transform()
    height = da.y.size
    width = da.x.size

    # Préparation temps
    df_time = pd.DataFrame({"date": pd.to_datetime(da.time.values)})
    df_time["year"] = df_time["date"].dt.year
    df_time["index"] = np.arange(len(df_time))

    # ===========================================================
    # 2) BOUCLE SUR COUCHES GÉOGRAPHIQUES
    # ===========================================================
    for layer in layers:
        print(f"\n🔹 Couche : {layer}")
        gdf = gpd.read_file(gpkg_file, layer=layer).to_crs(epsg=27572)

        if "id_ter" not in gdf.columns:
            print(f"⚠ id_ter manquant → couche ignorée")
            continue

        # ===========================================================
        # 3) BOUCLE SUR POLYGONES
        # ===========================================================
        for idx, row in gdf.iterrows():

            poly = row.geometry
            poly_id = row["id_ter"]

            # Rasterisation
            mask = rasterize(
                [mapping(poly)],
                out_shape=(height, width),
                transform=transform,
                fill=0,
                default_value=1,
                dtype="uint8"
            ).astype(bool)

            if not mask.any():
                print(f"⚠ Polygone {poly_id} sans pixels → ignoré")
                continue

            mask_flat = mask.ravel()

            vals = da.values.reshape(da.time.size, -1)[:, mask_flat]
            day_flags = vals > threshold  # True si dépassement seuil

            # ===========================================================
            # 4) CALCUL PAR PÉRIODE
            # ===========================================================
            for pname, (dstart, dend) in periods.items():
                print(f"   ▶ Période {pname}")

                start, end = pd.to_datetime(dstart), pd.to_datetime(dend)
                mask_period = (df_time["date"] >= start) & (df_time["date"] <= end)
                df_period = df_time[mask_period]

                if df_period.empty:
                    continue

                idx_sel = df_period["index"].values
                day_flags_period = day_flags[idx_sel, :]

                # 1 si au moins UN pixel dépasse le seuil
                days_over = day_flags_period.any(axis=1).astype(int)

                df_period = df_period.copy()
                df_period["days_over"] = days_over

                # Nombre de jours > seuil par année
                annual = df_period.groupby("year")["days_over"].sum()

                # moyenne annuelle
                mean_days = annual.mean()

                all_results.append({
                    "id_ter": poly_id,
                    "period": pname,
                    "modalite": mod_name,
                    "valeur": float(mean_days),
                })


# ===========================================================
# 5) EXPORT FINAL
# ===========================================================
df_res = pd.DataFrame(all_results)
outfile = "resultats_CNRM-ESM2-1_hot_days.csv"
df_res.to_csv(outfile, index=False)

print("\n✔ Résultats enregistrés dans :", outfile)



### Calcul du nombre de nuits tropicales par an par période TRACC

In [ ]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
from rasterio.features import rasterize
from shapely.geometry import mapping


# ===========================================================
# 0) LISTE DES TRAITEMENTS À EFFECTUER
# ===========================================================
jobs = [
    {
        "nc_file": r"C:\_GLOBAL\R_D\hackathon_mf\data\models\tasminAdjust_FR-Metro_CNRM-ESM2-1_historical_r14i1p1f2_CNRM-MF_CNRM-ALADIN64E1_v1-r1_MF-CDFt-ANASTASIA-SAFRAN-1985-2014_day_19500101-20141231.nc",
        "var_name": "tasminAdjust",
        "threshold": 293.15,  # 35°C en Kelvin
        "mod_name": "mean_hot_nights_per_year_1950_2014",
        "periods": {
            "1985-2014": ("1985-01-01", "2014-12-31")
        }
    },
        {
        "nc_file": r"C:\_GLOBAL\R_D\hackathon_mf\data\models\tasminAdjust_FR-Metro_CNRM-ESM2-1_ssp370_r14i1p1f2_CNRM-MF_CNRM-ALADIN64E1_v1-r1_MF-CDFt-ANASTASIA-SAFRAN-1985-2014_day_20150101-21001231.nc",
        "var_name": "tasminAdjust",
        "threshold": 293.15,  # 35°C en Kelvin
        "mod_name": "mean_hot_nights_per_year_ssp370_2015_2100",
        "periods": {
            "2037-2056": ("2037-01-01", "2056-12-31"),
            "2063-2082": ("2063-01-01", "2082-12-31"),
        }
    },
{
        "nc_file": r"C:\_GLOBAL\R_D\hackathon_mf\data\models\tasminAdjust_FR-Metro_CNRM-ESM2-1_ssp585_r1i1p1f2_CNRM-MF_CNRM-ALADIN64E1_v1-r1_MF-CDFt-ANASTASIA-SAFRAN-1985-2014_day_20150101-21001231.nc",
        "var_name": "tasminAdjust",
        "threshold": 293.15,  # 35°C en Kelvin
        "mod_name": "mean_hot_nights_per_ssp585_year_2015_2100",
        "periods": {
            "2032-2051": ("2032-01-01", "2051-12-31"),
            "2052-2061": ("2052-01-01", "2061-12-31"),
            "2072-2091": ("2072-01-01", "2091-12-31"),
        }
    },
]

# couches géographiques
gpkg_file = r"C:\_GLOBAL\R_D\hackathon_mf\data\supra_admin_20251202_093135.gpkg"
import fiona

layers = fiona.listlayers(gpkg_file)

# ===========================================================
# 1) BOUCLE GENERALE SUR LES FICHIERS
# ===========================================================
all_results = []

for job in jobs:
    print("\n===================================================")
    print(f"📌 Traitement fichier : {job['nc_file']}")
    print("===================================================")

    nc_file   = job["nc_file"]
    var_name  = job["var_name"]
    threshold = job["threshold"]
    periods   = job["periods"]
    mod_name  = job["mod_name"]

    # -----------------------------
    # CHARGEMENT NETCDF
    # -----------------------------
    ds = xr.open_dataset(nc_file)
    da = ds[var_name]

    transform = ds.rio.transform()
    height = da.y.size
    width = da.x.size

    # Préparation temps
    df_time = pd.DataFrame({"date": pd.to_datetime(da.time.values)})
    df_time["year"] = df_time["date"].dt.year
    df_time["index"] = np.arange(len(df_time))

    # ===========================================================
    # 2) BOUCLE SUR COUCHES GÉOGRAPHIQUES
    # ===========================================================
    for layer in layers:
        print(f"\n🔹 Couche : {layer}")
        gdf = gpd.read_file(gpkg_file, layer=layer).to_crs(epsg=27572)

        if "id_ter" not in gdf.columns:
            print(f"⚠ id_ter manquant → couche ignorée")
            continue

        # ===========================================================
        # 3) BOUCLE SUR POLYGONES
        # ===========================================================
        for idx, row in gdf.iterrows():

            poly = row.geometry
            poly_id = row["id_ter"]

            # Rasterisation
            mask = rasterize(
                [mapping(poly)],
                out_shape=(height, width),
                transform=transform,
                fill=0,
                default_value=1,
                dtype="uint8"
            ).astype(bool)

            if not mask.any():
                print(f"⚠ Polygone {poly_id} sans pixels → ignoré")
                continue

            mask_flat = mask.ravel()

            vals = da.values.reshape(da.time.size, -1)[:, mask_flat]
            day_flags = vals > threshold  # True si dépassement seuil

            # ===========================================================
            # 4) CALCUL PAR PÉRIODE
            # ===========================================================
            for pname, (dstart, dend) in periods.items():
                print(f"   ▶ Période {pname}")

                start, end = pd.to_datetime(dstart), pd.to_datetime(dend)
                mask_period = (df_time["date"] >= start) & (df_time["date"] <= end)
                df_period = df_time[mask_period]

                if df_period.empty:
                    continue

                idx_sel = df_period["index"].values
                day_flags_period = day_flags[idx_sel, :]

                # 1 si au moins UN pixel dépasse le seuil
                days_over = day_flags_period.any(axis=1).astype(int)

                df_period = df_period.copy()
                df_period["days_over"] = days_over

                # Nombre de jours > seuil par année
                annual = df_period.groupby("year")["days_over"].sum()

                # moyenne annuelle
                mean_days = annual.mean()

                all_results.append({
                    "id_ter": poly_id,
                    "period": pname,
                    "modalite": mod_name,
                    "valeur": float(mean_days),
                })


# ===========================================================
# 5) EXPORT FINAL
# ===========================================================
df_res = pd.DataFrame(all_results)
outfile = "resultats_CNRM-ESM2-1_hot_nights.csv"
df_res.to_csv(outfile, index=False)

print("\n✔ Résultats enregistrés dans :", outfile)

